In [1]:
import wordstats
json_str = wordstats.google_wordstats_json_daily('парфюм')

json_str

'[{"date":"2021-03-31","value":52},{"date":"2021-04-01","value":62},{"date":"2021-04-02","value":81},{"date":"2021-04-03","value":73},{"date":"2021-04-04","value":73},{"date":"2021-04-05","value":60},{"date":"2021-04-06","value":61},{"date":"2021-04-07","value":81},{"date":"2021-04-08","value":71},{"date":"2021-04-09","value":78},{"date":"2021-04-10","value":67},{"date":"2021-04-11","value":56},{"date":"2021-04-12","value":66},{"date":"2021-04-13","value":55},{"date":"2021-04-14","value":48},{"date":"2021-04-15","value":64},{"date":"2021-04-16","value":45},{"date":"2021-04-17","value":100},{"date":"2021-04-18","value":77},{"date":"2021-04-19","value":62},{"date":"2021-04-20","value":60},{"date":"2021-04-21","value":58},{"date":"2021-04-22","value":53},{"date":"2021-04-23","value":73},{"date":"2021-04-24","value":71},{"date":"2021-04-25","value":63},{"date":"2021-04-26","value":59},{"date":"2021-04-27","value":51},{"date":"2021-04-28","value":51},{"date":"2021-04-29","value":43},{"date"

In [19]:
import pandas as pd

stats_df = pd.read_json(json_str, orient='records')

stats_df = stats_df.groupby('date').max('value')

stats_df

,value
date,
2021-03-31,52
2021-04-01,62
2021-04-02,81
2021-04-03,73
2021-04-04,73
...,...
2023-03-24,58
2023-03-25,66
2023-03-26,57


In [15]:
import gtrend
from pytrends.request import TrendReq

# pytrends = TrendReq()
pytrends = TrendReq(hl='ru-RU', tz=-180)
# pytrends = TrendReq(hl='en-US', tz=360, timeout=(10,25), retries=2, backoff_factor=0.1)


kw_list = ["Blockchain"]
pytrends.build_payload(kw_list, cat=0, timeframe='today 5-y', geo='', gprop='').interest_over_time()


# gtrend.get_daily_trend(pytrends, geo='RU', keyword='парфюм', tz=-180, start='2023-01-01', end='2023-04-01')
# gtrend.get_daily_trend(pytrends, keyword='парфюм', start='2023-01-01', end='2023-01-03', verbose=True, sleep=1)

TooManyRequestsError: The request failed: Google returned a response with code 429

In [224]:
import plotly.express as px

enter_df = pd.read_csv('../data/marking/enter-aggregate.csv')
leave_df = pd.read_csv('../data/marking/leave-aggregate.csv')

stats_ya = pd.read_json('./yandex-wordstats/парфюм.json')


enter_df['enter_cnt'] = enter_df['enter_cnt'] / enter_df['enter_cnt'].max()
leave_df['leave_cnt'] = leave_df['leave_cnt'] / leave_df['leave_cnt'].max()

stats_df_new = stats_df.loc[(stats_df.index >= enter_df['dt'].min()) & (stats_df.index <= enter_df['dt'].max())]
stats_ya = stats_ya.loc[(stats_ya['date'] >= enter_df['dt'].min()) & (stats_ya['date'] <= enter_df['dt'].max())]

stats_df_new['value'] = stats_df_new['value'] / stats_df_new['value'].max()
stats_ya['value'] = stats_ya['totalCount'] / stats_ya['totalCount'].max()

enter_df['enter_cnt'] = enter_df['enter_cnt'].rolling(window=10, center=True).mean()
leave_df['leave_cnt'] = leave_df['leave_cnt'].rolling(window=10, center=True).mean()
stats_df_new['value'] = stats_df_new['value'].rolling(window=10, center=True).mean()

stats_ya = stats_ya.drop_duplicates(subset='date').sort_values('date')

stats_ya = stats_ya.loc[stats_ya['value'] != 0]
stats_ya = stats_ya.set_index('date')
stats_ya = stats_ya.resample('D').interpolate('linear', order=2)


# stats_ya['value'] = stats_df_new['value'].rolling(window=10, center=True).mean()

fig = px.line(stats_df_new, y="value", title='google')

fig.add_scatter(x=enter_df['dt'], y=enter_df['enter_cnt'], name='enter')

fig.add_scatter(x=leave_df['dt'], y=leave_df['leave_cnt'], name='leave')

fig.add_scatter(x=stats_ya.index, y=stats_ya['value'], name='yandex')

fig.show()
#
# print(pd.Series(stats_df_new['value'].values).corr(leave_df['leave_cnt'], method='pearson'))

# stats_ya

# px.line(stats_ya, y="value", title='google')

# px.line()

# stats_ya

max_lag = 100

# leave_df['dt'] = leave_df['dt'].shift(100)

google_values = []
ya_values = []

for lag in range(max_lag):
    val_leave = pd.Series(stats_df_new['value'].values).corr(leave_df.shift(50 - lag)['leave_cnt'])
    val_enter = pd.Series(stats_df_new['value'].values).corr(enter_df.shift(50 - lag)['enter_cnt'])
    google_values.append({'val_leave':val_leave, 'val_enter':val_enter, 'lag': 50 - lag})

    val_leave = pd.Series(stats_ya['value'].values).corr(leave_df.shift(50 - lag)['leave_cnt'])
    val_enter = pd.Series(stats_ya['value'].values).corr(enter_df.shift(50 - lag)['enter_cnt'])
    ya_values.append({'val_leave':val_leave, 'val_enter':val_enter, 'lag': 50 - lag})
#
# leave_df.shift(10)

# lagged_correlation = [stats_df_new['value'].corr(enter_df['enter_cnt'].shift(-t) for t in range(max_lag))]
#
# lagged_correlation

# pd.Series(values).max()
# pd.Series(values).mean()
google_df = pd.DataFrame(google_values)

px.line(google_df, x='lag', y=['val_leave', 'val_enter'], title='google')

C:\Users\criit\AppData\Local\Temp\ipykernel_14012\2411162411.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\criit\AppData\Local\Temp\ipykernel_14012\2411162411.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [227]:
ya_values

ya_df = pd.DataFrame(ya_values)

px.line(ya_df, x='lag', y=['val_leave', 'val_enter'], title='yandex')

# pd.Series(stats_ya['value'].values).corr(leave_df['leave_cnt'])